In [1]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd
import numpy as np

In [2]:

cookies = {
    'ccsid': '668-8707266-8901325',
    '__qca': 'P0-1002348396-1661184067604',
    'u': 'dtrNCBpDaLJaRutT2DQLSih0IUZ5EEkvU0E4PqVsLckLY2l8',
    'p': 'ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd',
    'likely_has_account': 'true',
    'logged_out_browsing_page_count': '2',
    'locale': 'en',
    'srb_8': '0_wl',
    '_session_id2': 'ebd665748c8c051639813d602272ce19',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9,id;q=0.8',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # Requests sorts cookies= alphabetically
    # 'Cookie': 'ccsid=668-8707266-8901325; __qca=P0-1002348396-1661184067604; u=dtrNCBpDaLJaRutT2DQLSih0IUZ5EEkvU0E4PqVsLckLY2l8; p=ZqHh-ArV88dYkVHfgapFDgVJRXU5lLzNc4VfzdNPfI5ZMVgd; likely_has_account=true; logged_out_browsing_page_count=2; locale=en; srb_8=0_wl; _session_id2=ebd665748c8c051639813d602272ce19',
    'If-None-Match': 'W/"23f0f18490caaa9a7080ce67914de66e"',
    'Referer': 'https://www.goodreads.com/shelf/show/horror?page=2',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Chromium";v="106", "Google Chrome";v="106", "Not;A=Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

# params = {
#     'page': '1',
# }

# response = requests.get('https://www.goodreads.com/shelf/show/horror', cookies=cookies, headers=headers)


In [56]:
def clean_publish_year(raw_year):
    if type(raw_year) is None:
        return "Missing"
    result = re.search(r"(\b\d+)$", raw_year)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_rating(raw_rating):
    if type(raw_rating) is None:
        return 0
    result = re.search(r"(\b\d+.\b\d+)", raw_rating)
    if result == None:
        return 0
    else:
        return result.group(1)

def clean_raters(raw_raters):
    if type(raw_raters) is None:
        return 0
    result = re.search(r"((?:\d+,?\d+)*) ratings", raw_raters)
    if result == None:
        return 0
    else:
        return result.group(1)
    
def clean_genres(raw_genres):
    genreList = []
    if raw_genres == None:
        return "Missing"
    else:
        for genre in raw_genres:
            if  genre == None:
                continue
            else:
                genreList.append(genre.find("a").get_text())
        return genreList
# def clean_readers(raw_readers):
#     result = re.search(r"(\b\d+)", raw_readers)
#     if result == None:
#         return "None"
#     else:
#         return result.group(1)

def solve_nonetype_error(value):
    if value is not None:
        return value.text.strip()
    else:
        return "Missing"



In [71]:

books = []
for page in range(1, 26):
    response = requests.get("https://www.goodreads.com/shelf/show/horror?page=" + str(page), cookies = cookies, headers=headers)
    html_soup = BeautifulSoup(response.text, 'html5lib')
    book_container = html_soup.find_all('div', class_ = "elementList")
    for book in book_container:
        bookTitle = solve_nonetype_error(book.find('a', class_ = "bookTitle"))
       
        bookAuthor =solve_nonetype_error(book.find('a', class_ = "authorName"))

        # After(use comments)
        year = clean_publish_year(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        rating = clean_rating(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))
        raters = str(clean_raters(solve_nonetype_error(book.find('span', class_ = "greyText smallText")))).replace(",", "")
        int(raters)
        # # readerNum = clean_readers(book.find('span', class_ = "greyText statistic").text.strip())
        
        # Before (use comments)
        # year = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # rating = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # raters = solve_nonetype_error(book.find('span', class_ = "greyText smallText"))
        # readerNum = book.find('span', class_ = "greyText statistic").text.strip()

        link = book.a['href']
        completeLink = "https://www.goodreads.com" + link
        getGenre = requests.get(completeLink,  cookies = cookies, headers=headers)
        genreParsing = BeautifulSoup(getGenre.text, 'html5lib')
        genres = clean_genres(genreParsing.findAll("div", class_=  "elementList", style = None))
        genreStr = ",".join(genres)
        book_data = [bookTitle, bookAuthor, year, rating, raters, genreStr]
        books.append(book_data)
books = pd.DataFrame(books, columns=["Title", "Author", "Year", "Rating", "Raters", "Genres"])
books.head()

KeyboardInterrupt: 

In [69]:
books.to_csv("HorrorTestWithGenres.csv", index = False)
